# Script for single input - single output txn -> Just input the transaction hex and run the cells, the script will calculate the private key for you!

In [24]:
import embit

from binascii import b2a_base64
from hashlib import sha256

from embit import bip32, compact, ec
from embit.bip32 import HDKey
from embit.descriptor import Descriptor
from embit.networks import NETWORKS
from embit.util import secp256k1

from embit.util.key import SECP256K1
from embit.util import key

from blockcypher import get_address_full
from blockcypher import get_transaction_details

from tinyec.ec import SubGroup, Curve
import numpy as np
import hashlib
import libnum
import bech32
import binascii
import hmac 


from cryptos import *

import ecdsa
from ecdsa import SigningKey, SECP256k1
import hashlib
import base58
from ecdsa.util import sigencode_der, sigdecode_der

from embit.util import secp256k1
from embit.util import py_secp256k1

import embit

from binascii import b2a_base64
from hashlib import sha256

from embit import bip32, compact, ec
from embit.bip32 import HDKey
from embit.descriptor import Descriptor
from embit.networks import NETWORKS
from embit.util import secp256k1

## Defining the neccessary functions

In [25]:
def double_sha256(data):
    
    in_bytes = bytes.fromhex(data)
    
    # First SHA256 hash
    first_hash = hashlib.sha256(in_bytes).digest()

    # Second SHA256 hash
    second_hash = hashlib.sha256(first_hash).digest()

    return second_hash.hex()

In [26]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1 
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def mod_inv(a,n = p): #Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1,0
    low, high = a%n,n
    
    while low > 1:
        ratio = high//low
        nm, new = hm-lm*ratio, high-low*ratio
        lm, low, hm, high = nm, new, lm, low
        
    return lm % n

In [27]:
SECP256K1_ORDER = n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def deterministic_k(secret, z, extra_data=None):
    # RFC6979, optimized for secp256k1
    k = b"\x00" * 32
    v = b"\x01" * 32
    if z > SECP256K1_ORDER:
        z -= SECP256K1_ORDER
    z_bytes = z.to_bytes(32, "big")
    secret_bytes = secret.to_bytes(32, "big") #because sha256(pub_key is always in bytes)
    if extra_data is not None:
        z_bytes += extra_data
    k = hmac.new(k, v + b"\x00" + secret_bytes + z_bytes, "sha256").digest()
    v = hmac.new(k, v, "sha256").digest()
    k = hmac.new(k, v + b"\x01" + secret_bytes + z_bytes, "sha256").digest()
    v = hmac.new(k, v, "sha256").digest()
    while True:
        v = hmac.new(k, v, "sha256").digest()
        candidate = int.from_bytes(v, "big")
        if candidate >= 1 and candidate < SECP256K1_ORDER:
            return candidate
        k = hmac.new(k, v + b"\x00", "sha256").digest()
        v = hmac.new(k, v, "sha256").digest()

In [28]:
def calc_scriptcode(addr):# calculate segwit input address scriptpubkey 
    
    hrp = addr[0:2]
    address_decoded = bech32.decode(hrp, addr)

    byte_string = bytes(address_decoded[1])
    hex_string = binascii.hexlify(byte_string).decode('utf-8')
    scriptpubkey = '19' + '76' + 'a9' + '14' + hex_string + '88' + 'ac' 

    return scriptpubkey

In [7]:
def calc_amount(val):
    amount_sats = val
    amount_sats_hex = hex(amount_sats)[2:] 


    if len(amount_sats_hex) % 2 != 0:
        amount_sats_hex = '0' + amount_sats_hex

    little_endian_bytes = bytes.fromhex(amount_sats_hex)[::-1]
    little_endian_padded = little_endian_bytes.ljust(8, b'\x00')

    amount = little_endian_padded.hex()
    
    return amount

In [29]:
# this function worked fine when tested with this particular txn... 
def calc_uncompressed_from_compressed(compressed_key):

    def pow_mod(x, y, z):
        "Calculate (x ** y) % z efficiently."
        number = 1
        while y:
            if y & 1:
                number = number * x % z
            y >>= 1
            x = x * x % z
        return number

    p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
    #compressed_key = '02581d9dc848fa707abc32329ab44caf5607583ec115664bb9b49ea562651fff71'
    y_parity = int(compressed_key[:2]) - 2
    x = int(compressed_key[2:], 16)
    a = (pow_mod(x, 3, p) + 7) % p
    y = pow_mod(a, (p+1)//4, p)
    if y % 2 != y_parity:
        y = -y % p
    uncompressed_key = '04{:x}{:x}'.format(x, y)
    
    return uncompressed_key

## Fetching neccessary info from BlockCypher API 

In [30]:
# input the txn. hash here 
txn_hash = input(str())

d882925ae657bcf2ee252ccfdffdf46fa87309168099af20727ae35833267414


In [31]:
txn_details = get_transaction_details(txn_hash, include_hex=True)

In [12]:
txn_details

{'block_hash': '00000000000000000003cee05b2534d58aba08caf02f8500e77811fa71ec4032',
 'block_height': 826395,
 'block_index': 2966,
 'hash': 'd882925ae657bcf2ee252ccfdffdf46fa87309168099af20727ae35833267414',
 'hex': '02000000000101f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db30200000000ffffffff016423000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc80247304402203193c5f490cb13e4c3cb2f6cbbb4d0c6af632ff8efa7ef12dd8bc5249dc5f50f0220334bdf2a3db5f98ae54ddb4bf90e83ff0874c51ed5a3a6b10ad3fddd94ec396f012102581d9dc848fa707abc32329ab44caf5607583ec115664bb9b49ea562651fff7100000000',
 'addresses': ['bc1qfmsav54qgj8af49flwaymla7a56nv5gszss2u7',
  'bc1ql7jqzp8wc2xp3ygusql3yplcldkej07gk7ga9n'],
 'total': 9060,
 'fees': 4280,
 'size': 191,
 'vsize': 110,
 'preference': 'low',
 'relayed_by': '147.135.136.241:8333',
 'confirmed': datetime.datetime(2024, 1, 19, 9, 58, 13, tzinfo=tzutc()),
 'received': datetime.datetime(2024, 1, 19, 9, 55, 24, 824000, tzinfo=tzutc()),
 'ver': 2,
 'do

In [32]:
# code to remove the unneccessary stuff from txn hex

full_txn_details_hex = txn_details['hex']
txn_details_hex = full_txn_details_hex[:8]+ full_txn_details_hex[12:160] + full_txn_details_hex[374:]

In [33]:
ver = txn_details_hex[0:8]
prevouts = txn_details_hex[10:74] + txn_details_hex[74:82]
seq = txn_details_hex[84:92]
outpoint = prevouts
input_addr = txn_details['inputs'][0]['addresses'][0]

scriptcode = calc_scriptcode(input_addr)
amount = calc_amount(txn_details['inputs'][0]['output_value']) 

output = txn_details_hex[94:110] + txn_details_hex[110:112] + txn_details_hex[112:156]
locktime = txn_details_hex[-8:] 
hashtype = '01000000'

In [34]:
prevouts_hash = double_sha256(prevouts)
seq_hash = double_sha256(seq)
outputs_hash = double_sha256(output)

In [35]:
pre_image = ver + prevouts_hash + seq_hash + outpoint + scriptcode + amount + seq + outputs_hash + locktime + hashtype
print(pre_image)

020000005600a8e58a8f4abe6265c31a13b42fda89772d4b7c953d6d39994de9ffb3927f3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044f7237c3a43b1a61da905fec85a3bba53ce0c6fd747168b855cd848ab4a992db3020000001976a9144ee1d652a0448fd4d4a9fbba4dffbeed3536511088ac1c34000000000000ffffffff32549df5abff5731231eaad1d373b5808b59820b3c4281cc0dbadc296c6f94700000000001000000


In [36]:
pre_image_hash_to_sign = double_sha256(pre_image) 
pre_image_hash_to_sign_bytes = bytes.fromhex(pre_image_hash_to_sign)

msghash = pre_image_hash_to_sign_bytes
msghash_int = int.from_bytes(msghash, "big")

compressed_pub_key = txn_details['inputs'][0]['witness'][1]

uncompressed_pub_key = calc_uncompressed_from_compressed(compressed_pub_key)
uncompressed_pub_key_bytes = bytes.fromhex(uncompressed_pub_key)

pub_hash_int = int.from_bytes(hashlib.sha256(uncompressed_pub_key_bytes).digest(), "big")

signature = txn_details['inputs'][0]['witness'][0]

## Private key calculation logic

In [37]:
r_len = 2*int(signature[6:8], 16)
r_hex = signature[8:(8+r_len)]

s_len = 2*int(signature[(8+r_len+2):(8+r_len+2+2)], 16)
s_hex = signature[(8+r_len+2+2):(8+r_len+2+2+s_len)]

In [38]:
r_int = int(r_hex, 16)
s_int = int(s_hex, 16)

k = deterministic_k(pub_hash_int, msghash_int) 

In [39]:
calc_prv = (mod_inv(r_int, n)*(k*s_int - msghash_int)) % n 
print("Calculated private key (hex):", hex(calc_prv))

Calculated private key (hex): 0xce1539c8fe4a767d3a82329e806493c8034cd946371103223353ca82605ec51c
